In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
datax = [
    [[0.1517, 0.5443, 0.1664, 0.2968, 0.1047, 0.2303, 0.7809, 0.8742]],
    [[0.6482, 0.8415, 0.3437, 0.5621, 0.8318, 0.5978, 0.2179, 0.5826]],
    [[0.0959, 0.5683, 0.8281, 0.2220, 0.7972, 0.3602, 0.9269, 0.3220]],
    [[0.2017, 0.4027, 0.9019, 0.3093, 0.2855, 0.0226, 0.3115, 0.5141]],
]

# data shape
batch = 4
length = 8

# conv shape
k_size = 3
chan_in = 1
chan_out = 2

params = np.random.rand(k_size, chan_in, chan_out)


In [3]:
np_input = np.array(datax).reshape(batch, length, chan_in)
th_input = torch.Tensor(datax)

In [4]:
from lamp.conv import Conv1D

m1 = Conv1D(k_size, chan_in, chan_out)
output = m1.forward(np_input)
np.round_(output.reshape(batch, chan_out, -1), decimals=4)

array([[[ 0.0124,  0.0392, -0.0249,  0.0099,  0.0034,  0.0195],
        [-0.0164,  0.0101, -0.0329,  0.0554, -0.0156,  0.0917]],

       [[ 0.0015,  0.0522, -0.0397,  0.0266, -0.0274,  0.0717],
        [-0.0094,  0.0687, -0.0094,  0.0237, -0.039 ,  0.0275]],

       [[-0.019 ,  0.0799,  0.0097,  0.0471, -0.0572,  0.0325],
        [ 0.0121,  0.064 , -0.0564,  0.0496,  0.0159,  0.0657]],

       [[-0.0341,  0.0701,  0.014 ,  0.062 , -0.0289,  0.0068],
        [ 0.0025,  0.0095, -0.0242,  0.0098, -0.0126,  0.0481]]])

In [5]:
m2 = nn.Conv1d(chan_in, chan_out, k_size)
output = m2(th_input)
output

tensor([[[0.4631, 0.4323, 0.4794, 0.4581, 0.3771, 0.3392],
         [0.5553, 0.6004, 0.4148, 0.4326, 0.6450, 1.0086]],

        [[0.3994, 0.3689, 0.3463, 0.3638, 0.4167, 0.3824],
         [0.9951, 0.9316, 0.9339, 1.0613, 0.9036, 0.7716]],

        [[0.3601, 0.4234, 0.3357, 0.4210, 0.3127, 0.4158],
         [0.8260, 0.9012, 0.9672, 0.7925, 1.0765, 0.9003]],

        [[0.3475, 0.4163, 0.4107, 0.4851, 0.4482, 0.4216],
         [0.8236, 0.9015, 0.8230, 0.4372, 0.4224, 0.5340]]],
       grad_fn=<SqueezeBackward1>)

In [6]:
from lamp import MaxPool1D

m3 = MaxPool1D(k_size, stride=1)
output = m3.forward(np_input)
output.reshape(batch, chan_in, -1)

array([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

       [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

       [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

       [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [7]:
m4 = nn.MaxPool1d(k_size, stride=1)
output = m4(th_input)
output

tensor([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

        [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

        [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

        [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [15]:
input = m4(th_input)
# With default parameters
m6 = nn.Flatten()
output = m6(input)
output.size(), output

(torch.Size([4, 6]),
 tensor([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
         [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978],
         [0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269],
         [0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]))

In [16]:
from lamp import Flatten

input = m3.forward(np_input)

m5 = Flatten()
output = m5.forward(input)
output.shape, output

((4, 6),
 array([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
        [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978],
        [0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269],
        [0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]))